# 점수화 하기

In [11]:
# ==============================
# 1-1. 라이브러리 불러오기
# ==============================
import json  # JSON 파일 읽기/쓰기
import numpy as np  # 수치 연산
from sklearn.linear_model import LinearRegression  # 선형 회귀 모델

# ==============================
# 1-2. JSON 파일 경로 설정
# ==============================
json_path = '/Users/laxdin24/Documents/GitHub/MS_AI_SCHOOL_6/Project All/3차프로젝트/output_Pushup_test4.json'

# ==============================
# 2. 회귀선 적합 및 궤도 일관성 분석
# ==============================
def linear_fit_error(points):
    """
    주어진 2D 좌표에 대해 x축 기준 선형 회귀선 적합
    → 실제 y와 예측된 y의 오차(절대값 평균)와 기울기 반환
    """
    X = np.array(points[:, 0]).reshape(-1, 1)  # x좌표
    y = points[:, 1]  # y좌표
    model = LinearRegression().fit(X, y)
    y_pred = model.predict(X)
    error = np.mean(np.abs(y - y_pred))  # y 예측값과 실제값 차이의 평균 (궤도의 일관성 측정)
    slope = model.coef_[0]  # 회귀 직선의 기울기
    return model, error, slope

# ==============================
# 3. 어깨-골반의 동기화 오차 계산
# ==============================
def sync_error(shoulder_pts, hip_pts):
    """
    어깨와 골반의 프레임 간 변화량 차이 계산
    → x축, y축의 이동량 차이를 프레임 단위로 비교 후 평균
    → 동기화가 잘 되면 두 변화량이 유사해야 함
    """
    dx_s = np.diff(shoulder_pts[:, 0])  # 어깨 x 변화량
    dy_s = np.diff(shoulder_pts[:, 1])  # 어깨 y 변화량
    dx_h = np.diff(hip_pts[:, 0])       # 골반 x 변화량
    dy_h = np.diff(hip_pts[:, 1])       # 골반 y 변화량
    error_x = np.mean(np.abs(dx_s - dx_h))  # x 변화량 차이 평균
    error_y = np.mean(np.abs(dy_s - dy_h))  # y 변화량 차이 평균
    return error_x, error_y

# ==============================
# 4. 평가 함수
# ==============================
def evaluate_pushup(json_path):
    # (1) JSON 파일에서 어깨와 골반 좌표를 읽음
    with open(json_path, "r") as f:
        data = json.load(f)

    shoulder = np.array([f["shoulder_xy"] for f in data])  # 어깨 좌표 시퀀스
    hip = np.array([f["hip_xy"] for f in data])            # 골반 좌표 시퀀스

    # (2) 이동 평균 필터 (노이즈 제거) — 현재는 비활성화
    # shoulder = moving_average(shoulder, k=5)
    # hip = moving_average(hip, k=5)

    # (3) 궤도 일관성: 어깨/골반 각각에 대해 회귀선 적합 및 오차 계산
    _, shoulder_err, shoulder_slope = linear_fit_error(shoulder)
    _, hip_err, hip_slope = linear_fit_error(hip)

    # (4) 어깨-골반 간 프레임별 이동 동기화 오차
    sync_x, sync_y = sync_error(shoulder, hip)

    # (5) 기울기 차이: 회귀선의 방향이 어깨와 골반이 얼마나 다른지
    slope_diff = abs(shoulder_slope - hip_slope)

    # (6) 점수 계산 (감점식)
    #   - 오차와 차이가 클수록 감점
    #   - 각 요소에 가중치 0.1 부여
    raw_score = 100
    penalty = shoulder_err + hip_err + sync_x + sync_y + slope_diff
    final_score = max(0, raw_score - penalty)  # 0점 미만 방지

    # (7) 결과 리턴
    return {
        "shoulder_err": round(shoulder_err, 3),
        "hip_err": round(hip_err, 3),
        "sync_x": round(sync_x, 3),
        "sync_y": round(sync_y, 3),
        "slope_diff": round(slope_diff, 3),
        "score": round(final_score, 2)
    }

# ==============================
# 5. 실행 진입점
# ==============================
if __name__ == "__main__":
    # 지정된 JSON 파일을 평가하고 결과 출력
    print(evaluate_pushup(json_path))

{'shoulder_err': 18.966, 'hip_err': 12.124, 'sync_x': 0.482, 'sync_y': 0.878, 'slope_diff': 0.749, 'score': 66.8}


## 측정된 shoulder_xy, hip_xy 좌표값을 정규화해서 점수산정하기

In [6]:
import json
import numpy as np
from sklearn.linear_model import LinearRegression
from glob import glob
import os
import pandas as pd

# ==============================
# 1. 좌표 정규화 함수
# ==============================
def normalize_coords(coords, width, height):
    """
    좌표를 해상도 기준으로 [0, 1] 범위로 정규화함
    → 해상도 차이를 제거하여 공정 비교 가능
    """
    return np.array([[x / width, y / height] for x, y in coords])

# ==============================
# 1.5 이동 평균 보정 함수 추가
# ==============================
def moving_average(points, k=5):
    """(x, y) 좌표 시퀀스를 k 프레임 기반 이동 평균 필터로 부드럽게"""
    if len(points) < k:
        return points
    smoothed = []
    for i in range(len(points)):
        start = max(0, i - k//2)
        end = min(len(points), i + k//2 + 1)
        window = points[start:end]
        avg_x = np.mean([p[0] for p in window])
        avg_y = np.mean([p[1] for p in window])
        smoothed.append([avg_x, avg_y])
    return np.array(smoothed)

# ==============================
# 2. 회귀선 분석 함수
# ==============================
def linear_fit_error(points):
    """
    x값 기준으로 선형 회귀 적합 후
    → 실제 y와 예측 y 차이의 평균 오차 + 기울기 반환
    """
    X = np.array(points[:, 0]).reshape(-1, 1)
    y = points[:, 1]
    model = LinearRegression().fit(X, y)
    y_pred = model.predict(X)
    error = np.mean(np.abs(y - y_pred))
    slope = model.coef_[0]
    return model, error, slope

# ==============================
# 3. 어깨-골반 동기화 분석
# ==============================
def sync_error(shoulder_pts, hip_pts):
    """
    각 프레임 간 Δx, Δy 이동량을 비교하여
    → 두 궤적이 얼마나 유사하게 움직였는지 계산
    """
    dx_s = np.diff(shoulder_pts[:, 0])
    dy_s = np.diff(shoulder_pts[:, 1])
    dx_h = np.diff(hip_pts[:, 0])
    dy_h = np.diff(hip_pts[:, 1])
    error_x = np.mean(np.abs(dx_s - dx_h))
    error_y = np.mean(np.abs(dy_s - dy_h))
    return error_x, error_y

# ==============================
# 4. 전체 점수 산정 함수
# ==============================
def evaluate_pushup(json_path):
    with open(json_path, "r") as f:
        data = json.load(f)

    # ⬛ 해상도 확인 (모든 프레임 동일하므로 첫 프레임 사용)
    width = data[0]["frame_width"]
    height = data[0]["frame_height"]

    # ⬛ 좌표 정규화
    shoulder = normalize_coords([f["shoulder_xy"] for f in data], width, height)
    hip = normalize_coords([f["hip_xy"] for f in data], width, height)

    # ✅ 이동 평균 적용
    shoulder = moving_average(shoulder, k=10)
    hip = moving_average(hip, k=10)

    # ⬛ 궤도 일관성 분석
    _, shoulder_err, shoulder_slope = linear_fit_error(shoulder)
    _, hip_err, hip_slope = linear_fit_error(hip)

    # ⬛ 동기화 오차 분석
    sync_x, sync_y = sync_error(shoulder, hip)

    # ⬛ 기울기 차이 분석
    slope_diff = abs(shoulder_slope - hip_slope)

    # ⬛ 점수 계산
    raw_score = 100
    penalty = (shoulder_err + hip_err) * 150 + (sync_x + sync_y) * 1000 + slope_diff * 5
    final_score = max(0, raw_score - penalty)


    return {
        "shoulder_err": round(shoulder_err, 4),
        "hip_err": round(hip_err, 4),
        "sync_x": round(sync_x, 4),
        "sync_y": round(sync_y, 4),
        "slope_diff": round(slope_diff, 4),
        "shoulder_slope": round(shoulder_slope, 4),
        "score": round(final_score, 2)
    }

# ==============================
# 5. 폴더 전체 평가 (파일명 포함)
# ==============================
def evaluate_all_json(folder_path):
    result_list = []
    json_files = sorted(glob(os.path.join(folder_path, "*.json")))

    for path in json_files:
        result = evaluate_pushup(path)
        result["file_name"] = os.path.basename(path)  # 🔹 파일 이름 추가
        result_list.append(result)

    # 🔹 파일명을 첫 번째 열로 DataFrame 구성
    df = pd.DataFrame(result_list)
    cols = ["file_name"] + [col for col in df.columns if col != "file_name"]
    return df[cols]

# ==============================
# 6. 실행 예시
# ==============================
if __name__ == "__main__":
    pd.set_option('display.width', 200)  # 혹은 300 이상
    folder_path = "/Users/laxdin24/Desktop/Output_json"
    df = evaluate_all_json(folder_path)
    print(df)

          file_name  shoulder_err  hip_err  sync_x  sync_y  slope_diff  shoulder_slope  score
0  Nomal_test1.json        0.0307   0.0346  0.0007   0.003      1.0497         -4.1781  81.26
